In [1]:
import nltk
import numpy as np
import collections
import re
import pandas as pd

from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import SnowballStemmer
from csv_helper import CSVHelper

nltk.download('stopwords')
nltk.download('wordnet')

data = CSVHelper.load_csv("Tweets_2016London.csv")

[nltk_data] Downloading package stopwords to /home/jeroen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jeroen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
starting for loop!


In [2]:
def tokenize(tweets):
    tknzr = TweetTokenizer()
    tokenized_tweets = []
    for tweet in tweets:
        tokenized_tweets.append(tknzr.tokenize(tweet))
    return tokenized_tweets

def remove_stopwords(tweets):
    stopwords =  nltk.corpus.stopwords.words('english')
    tweets_nostop = []
    for tweet in tweets:
        tweet_nostop = [w.lower() for w in tweet if w.lower() not in stopwords]
        tweets_nostop.append(tweet_nostop)
    return tweets_nostop

def filter_noise(tweets):
    tweets_filtered = []
    for tweet in tweets:
        tweet_filtered = [re.sub("[^a-zA-Z0-9]",'', w) for w in tweet if re.sub("[^a-zA-Z0-9]",'', w)!= '']
        tweets_filtered.append(tweet_filtered)
    return tweets_filtered

def remove_url(tweets):
    tweets_filtered = []
    for tweet in tweets:
        tweet_filtered = [re.sub(r'^https?:\/\/.*[\r\n]*', '', w) for w in tweet]
        tweets_filtered.append(tweet_filtered)
    return tweets_filtered

#we decided to use a lemmatizer instead of stemming 
def my_stem(tweets):
    lmt = WordNetLemmatizer()
    ps = PorterStemmer()
    tweets_stemmed = []
    for tweet in tweets:
        tweet_stemmed = [lmt.lemmatize(w) for w in tweet]
        tweets_stemmed.append(tweet_stemmed)
    return tweets_stemmed

In [68]:
all_tweets = tokenize(data)
all_tweets = remove_url(all_tweets)
all_tweets = filter_noise(all_tweets)
all_tweets = remove_stopwords(all_tweets)
all_tweets = my_stem(all_tweets)
print(all_tweets)

[['please', 'see', 'link', 'youtube', 'channel', 'darteh', 'ministry', 'please', 'like', 'share', 'subscribe'], ['northeastedl', 'lol', 'happens', 'put', 'woman', 'charge'], ['side', 'man', 'get', 'smash', 'show', 'shout', 'tv', 'swear', 'get', 'ladddd'], ['le', '1', 'month'], ['dunzocs', 'lived', '3', 'year', 'visit', 'la', 'every', 'month', 'home', 'second', 'home', 'tbh', 'bday', 'could'], ['described', 'met', 'heart', 'broken', 'im', 'touched'], ['thetwomikes', 'dcrosss', 'talksport', 'talkradio', 'ya', 'dozy', 'sausage'], ['sometimes', 'life', 'hard', 'dust', 'keep', 'going', 'nolimit'], ['dunzocs', 'end', 'happening'], ['tmp', '03', 'c', 'wind', '0mph', 'press', '10274', 'mb', 'cloud', '837', 'ft', 'rain', '00', 'mm', 'humidity', '86', 'fine', 'weather', 'ukweather'], ['themissing', 'cmaawards50', 'wembley', 'spur', 'thistimenextyear', '201611', '3', '0114', 'gmt', 'trndnl'], ['tgfone4', 'worldseries', 'real', 'madrid', 'andy', 'carroll', 'sissoko', '201611', '3', '0114', 'gmt', 

In [4]:
def termfreq(term, doc):
    #print("tweet:", doc, "length of tweet:", len(doc))
    length = len(doc)
    frequency = doc.count(term)
    return frequency/length

def count_doc_with_term(term, tweets):
    ctr = 0
    for tweet in tweets:
        if term in tweet:
            ctr = ctr + 1
    return ctr

#idf = amount_of_tweets/amount_of_tweets_with_word
def idf(term, tweets):
    num_docs = len(tweets)
    num_docs_with_word = count_doc_with_term(term, tweets)
    return np.log10(num_documents/num_docs_with_word)

def create_vocabulary(tweets):
    vocabulary = set()
    for tweet in tweets:
        words = [w for w in tweet]
        vocabulary.update(words)
    return list(vocabulary)

def compute_word_idf(tweets):
    vocabulary = create_vocabulary(tweets)
    word_idf = collections.defaultdict(lambda: 0)
    #for tweet in tweets:
    #    words = set(tweet)
    #    for w in words:
    #        word_idf[w] +=1        
    for w in vocabulary:
        word_idf[w] = idf(w, tweets)
    return word_idf
    
def tf_idf(term, doc, tweets):
    if doc != []:
        return termfreq(term, doc) * word_idf[term]
    else:
        return 0

https://nlpforhackers.io/tf-idf/

https://www.youtube.com/watch?v=4vT4fzjkGCQ&t=1s


http://billchambers.me/tutorials/2014/12/21/tf-idf-explained-in-python.html

In [ ]:
def tweet_features(tweets):
    vocabulary = create_vocabulary(tweets)
    tweet_features = np.empty(len(tweets), dtype=object)
    for i, tweet in enumerate(tweets):
        a = np.zeros(len(vocabulary))
        for j, word in enumerate(vocabulary):
            #print(j, word)
            a[j] = tf_idf(word, tweet, tweets)
        tweet_features[i] = a
    return tweet_features


test = [["this","a","a","is","sample"],["this","another","another","is","example","example","example"]]
test2 = [["new","times","york"],["new","post","york"],["angeles","los","times"]]


word_idf = compute_word_idf(all_tweets)
tweet_features = tweet_features(all_tweets)

feature_matrix = np.vstack(tweet_features)
pd.DataFrame(feature_matrix)

In [73]:
def cosine_distance(v1,v2):
    nomer = np.dot(v1, v2)
    denomer = np.dot(np.linalg.norm(v1), np.linalg.norm(v2))
    return nomer/denomer

In [78]:
#TODO from here
def calculate_distances(features):
    size = len(features)
    distances = np.zeros((size, size))
    for i in range(len(distances)):
        for j in range(len(distances)):
            distances[i,j] = cosine_distance(features[i], features[j])
    return distances

distances = calculate_distances(feature_matrix)

/home/jeroen/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


KeyboardInterrupt: 

In [77]:
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import cosine_similarity